In [1]:
from collections import deque
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

import gym
from tqdm.notebook import tqdm
import numpy as np
from typing import NamedTuple
from itertools import chain

In [2]:
from policy_generator.policy_instances.envs.simple_arena import ActionSpace

In [3]:
#device to run model on 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
class ObsSpace(NamedTuple):
    agent: np.ndarray
    agent_direction: int
    target: np.ndarray
    velocity: int

In [5]:
#Using a neural network to learn our policy parameters
class QLearningNetwork(nn.Module):
    #Takes in observations and outputs actions
    def __init__(self, observation_space, action_space, shape):
        super(QLearningNetwork, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(observation_space, shape),
            nn.ReLU(),
            nn.Linear(shape, shape),
            nn.ReLU(),
            nn.Linear(shape, shape),
            nn.ReLU(),
            nn.Linear(shape, action_space)
        )
    
    #forward pass
    def forward(self, x):
        return self.model(x)

class QBot:
    def __init__(self):
        self.model = None
        self.train_config = None
        
    def training_config(self, **kwargs):
        self.train_config = {
                'discount_factor': 0.923,
                'eps': 0.42,
                'eps_min': 0.075,
                'eps_decay': 0.96,
                'learning_rate': 0.053,
                'num_episodes': 50,
                'batch_size': 32,
                'network_shape': 128,
                }
        if kwargs:
            self.train_config.update(**kwargs)
            
#             b_params = {'discount_factor': 0.9232432057242249,
#                          'eps': 0.41987501329393667,
#                          'eps_min': 0.07487696385957002,
#                          'eps_decay': 0.9625856506789202,
#                          'learning_rate': 0.052947112503709155,
#                          'network_shape': 27}
            
    def _init_environment(self):
        #Make environment
        env = gym.make("policy_instances/SimpleArena-v0")

        network = QLearningNetwork(env.shape, env.action_space.n, self.train_config['network_shape']).to(DEVICE)

        loss_fn = nn.MSELoss()
        optimizer = optim.Adam(network.parameters(), lr=self.train_config['learning_rate'])
        return env, network, loss_fn, optimizer
    
    def train(self, verbose=0):
        env, network, loss_fn, optimizer = self._init_environment()
        
        scores = []

        memory = deque(maxlen=4000)

        for i in tqdm(range(self.train_config['num_episodes']), position=0, leave=True):
            eps = self.train_config['eps']
            state = env.reset()
            done = False
            eps *= self.train_config['eps_decay']
            score = 0
            actions_dist = []
            #while game not ended
            while not done:
                env.render()
                if isinstance(state, tuple):
                    state = state[0]
                #choose move with epsilon greedy
                if np.random.random() < eps:
                    #exploration
                    action = np.random.randint(0, env.action_space.n)
                else:
                    #exploitation
                    #use expand_dims here to add a dimension for input layer
                    action = select_action(network, np.expand_dims(state, axis=0))[0]
                    actions_dist.append(action)

                #execute move
                new_state, reward, done, _, _ = env.step(action)
                score += reward

                #memorize
                memory.append([np.expand_dims(state, axis=0), action, reward, np.expand_dims(new_state, axis=0), done])

                #update
                #instead of training every state, we train in batch_size
                if len(memory) > self.train_config['batch_size']:
                    #sample batch_size so model could be fit on any random states in memory not just the latest state
                    minibatch = random.sample(memory, self.train_config['batch_size'])
                    loss = 0

                    #iterate through the sampled batch
                    for b_state, b_action, b_reward, b_new_state, b_done in minibatch:
                        #if current game is done then target = reward cuz theres no future utility
                        if b_done:
                            target = b_reward
                        else:
                            #what we think the state's q_val should be, reward + discounted future utility
                            target = b_reward + self.train_config['discount_factor'] * select_action(network, b_new_state)[2]

                        #what we thought the current state's q_val should be
                        target_vector = select_action(network, b_state)[1]

                        #update the target_vector 
                        target_vector[0][b_action] = target

                        #instead of finding temporal difference between new q_val and old q_val, we train the model by giving it the new q_val
                        # and let the network do the updating 
                        #train the model with the batch
                        loss = loss_fn(select_action(network, b_state)[1], target_vector)
                        optimizer.zero_grad()
                        loss.backward()

                        step = optimizer.step()
#                     optimizer.zero_grad()
#                     loss.backward()

#                     step = optimizer.step()

                    #update epsilon
                    if eps > self.train_config['eps_min']:
                        eps *= self.train_config['eps_decay']

                #new state
                state = new_state
            if verbose > 0:
                print(score)
                print(list(zip(*np.unique(actions_dist, return_counts=True))))
            scores.append(score)
        return scores

            
def select_action(network, state):
    ''' Selects an action given current state
    Args:
    - network (Torch NN): network to process state
    - state (Array): Array of action space in an environment
    
    Return:
    - (int): action that is selected
    - (float): log probability of selecting that action given state and network
    '''
    #convert state to float tensor, add 1 dimension, allocate tensor on device
    state = ObsSpace(**state[0] if isinstance(state, (tuple, list, np.ndarray)) else state)
    unpack_state = list(chain(state.agent, state.target, [state.velocity, state.agent_direction]))
    state = torch.Tensor(unpack_state).float().unsqueeze(0).to(DEVICE)
    
    #use network to predict action probabilities
    q_vals = network(state)
    
    #sample an action using the probability distribution
    action = torch.argmax(q_vals)
    max_value = torch.max(q_vals)
    
    #return action
    return action.item(), q_vals, max_value

In [6]:
bot = QBot()
bot.training_config(**{"num_episodes": 200})
bot.train(1)

  0%|          | 0/200 [00:00<?, ?it/s]

-34
[(0, 6), (1, 40), (2, 7), (4, 29)]
26
[(0, 12), (1, 43), (2, 16), (3, 4), (4, 15)]
-36
[(0, 11), (1, 35), (2, 19), (3, 8), (4, 13)]
-28
[(0, 11), (1, 38), (2, 13), (3, 8), (4, 16)]
-40
[(0, 13), (1, 33), (2, 17), (3, 9), (4, 20)]


KeyboardInterrupt: 

In [16]:
import optuna

def objective(trial):
    bot = QBot()
    bot.training_config(**{
        'discount_factor': trial.suggest_float('discount_factor', 0.90, 0.99),
        'eps': trial.suggest_float('eps', 0.4, 0.6),
        'eps_min': trial.suggest_float('eps_min', 0.005, 0.1),
        'eps_decay': trial.suggest_float('eps_decay', 0.90, 0.99),
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1),
        'num_episodes': 20,
        'batch_size': 32,
        'network_shape': trial.suggest_int('network_shape', 16, 128),
                    })
    score = bot.train()
    return -1 * np.mean(score)

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params

[I 2022-09-19 12:08:52,772] A new study created in memory with name: no-name-36dc40b2-ab18-4de0-b16d-0e7bbcc87e24


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2022-09-19 12:12:12,123] Trial 0 finished with value: 333.0 and parameters: {'discount_factor': 0.961149283518651, 'eps': 0.5039940575634445, 'eps_min': 0.09187487028585081, 'eps_decay': 0.9368721683610204, 'learning_rate': 0.06780245382687256, 'network_shape': 46}. Best is trial 0 with value: 333.0.


  0%|          | 0/20 [00:00<?, ?it/s]

[I 2022-09-19 12:15:30,832] Trial 1 finished with value: 287.25 and parameters: {'discount_factor': 0.9204618347705573, 'eps': 0.5541758911752037, 'eps_min': 0.02966008613644321, 'eps_decay': 0.9052466827620687, 'learning_rate': 0.0023205613233050378, 'network_shape': 27}. Best is trial 1 with value: 287.25.


...

  0%|          | 0/20 [00:00<?, ?it/s]

[I 2022-09-19 16:02:46,627] Trial 76 finished with value: 265.9 and parameters: {'discount_factor': 0.9402241326062796, 'eps': 0.5424744667471213, 'eps_min': 0.022253502283242216, 'eps_decay': 0.961973286393732, 'learning_rate': 0.07479473831652023, 'network_shape': 79}. Best is trial 5 with value: 243.5.


  0%|          | 0/20 [00:00<?, ?it/s]

[W 2022-09-19 16:03:39,184] Trial 77 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "/home/beast/.local/lib/python3.8/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_685/1136845993.py", line 15, in objective
    score = bot.train()
  File "/tmp/ipykernel_685/1305696908.py", line 117, in train
    loss.backward()
  File "/home/beast/.local/lib/python3.8/site-packages/torch/_tensor.py", line 396, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
  File "/home/beast/.local/lib/python3.8/site-packages/torch/autograd/__init__.py", line 173, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt


KeyboardInterrupt: 

In [17]:
study.best_params

{'discount_factor': 0.9232432057242249,
 'eps': 0.41987501329393667,
 'eps_min': 0.07487696385957002,
 'eps_decay': 0.9625856506789202,
 'learning_rate': 0.052947112503709155,
 'network_shape': 27}

In [3]:
discount_factor = 0.95
eps = 0.5
eps_min = 0.01
eps_decay = 0.99
learning_rate = 0.8
num_episodes = 50
batch_size = 32

In [5]:
#Using a neural network to learn our policy parameters
class QLearningNetwork(nn.Module):
    
    #Takes in observations and outputs actions
    def __init__(self, observation_space, action_space):
        super(QLearningNetwork, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(observation_space, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, action_space)
        )
    
    #forward pass
    def forward(self, x):
        return self.model(x)

In [6]:
def select_action(network, state):
    ''' Selects an action given current state
    Args:
    - network (Torch NN): network to process state
    - state (Array): Array of action space in an environment
    
    Return:
    - (int): action that is selected
    - (float): log probability of selecting that action given state and network
    '''
    #convert state to float tensor, add 1 dimension, allocate tensor on device
    state = ObsSpace(**state[0] if isinstance(state, (tuple, list, np.ndarray)) else state)
    unpack_state = list(chain(state.agent, state.target, [state.velocity, state.agent_direction]))
    state = torch.Tensor(unpack_state).float().unsqueeze(0).to(DEVICE)
    
    #use network to predict action probabilities
    q_vals = network(state)
    
    #sample an action using the probability distribution
    action = torch.argmax(q_vals)
    max_value = torch.max(q_vals)
    
    #return action
    return action.item(), q_vals, max_value

In [7]:
#Make environment
env = gym.make("policy_instances/SimpleArena-v0")

network = QLearningNetwork(env.shape, env.action_space.n).to(DEVICE)

loss_fn = nn.MSELoss()
optimizer = optim.Adam(network.parameters(), lr=1e-3)

In [11]:
def check_availability():
    state = env.reset()
    print(select_action(network, state))
check_availability()

(1, tensor([[-49.4963, -48.6610, -49.2828, -51.9491, -49.0963]], device='cuda:0',
       grad_fn=<AddmmBackward0>), tensor(-48.6610, device='cuda:0', grad_fn=<MaxBackward1>))


In [9]:
scores = []

memory = deque(maxlen=4000)

for i in tqdm(range(num_episodes), position=0, leave=True):
    state = env.reset()
    done = False
    eps *= eps_decay
    score = 0
    actions_dist = []
    #while game not ended
    while not done:
        env.render()
        if isinstance(state, tuple):
            state = state[0]
        #choose move with epsilon greedy
        if np.random.random() < eps:
            #exploration
            action = np.random.randint(0, env.action_space.n)
        else:
            #exploitation
            #use expand_dims here to add a dimension for input layer
#             q_vals = model(state_torch)#.to('cpu').detach().numpy()
#             action = torch.argmax(q_vals).item()
            action = select_action(network, np.expand_dims(state, axis=0))[0]
            actions_dist.append(action)
        
        #execute move
        new_state, reward, done, _, _ = env.step(action)
        score += reward
        
        #modify reward so it scales with pole angle. Pole angle range [-0.418, 0.418]
        # reward = 1 - abs(state[2])/0.418
        
        #memorize
        memory.append([np.expand_dims(state, axis=0), action, reward, np.expand_dims(new_state, axis=0), done])
        
        #update
        #instead of training every state, we train in batch_size
        if len(memory) > batch_size:
            #sample batch_size so model could be fit on any random states in memory not just the latest state
            minibatch = random.sample(memory, batch_size)
            loss = 0
            
            #iterate through the sampled batch
            for b_state, b_action, b_reward, b_new_state, b_done in minibatch:
                #if current game is done then target = reward cuz theres no future utility
                if b_done:
                    target = b_reward
                else:
                    #what we think the state's q_val should be, reward + discounted future utility
                    target = b_reward + discount_factor * select_action(network, b_new_state)[2]
                
                #what we thought the current state's q_val should be
                target_vector = select_action(network, b_state)[1]
                
                #update the target_vector 
                target_vector[0][b_action] = target
                
                #instead of finding temporal difference between new q_val and old q_val, we train the model by giving it the new q_val
                # and let the network do the updating 
                #train the model with the batch
                loss += loss_fn(select_action(network, b_state)[1], target_vector)
                # optimizer.zero_grad()
                # loss.backward()
                
                # step = optimizer.step()
            optimizer.zero_grad()
            loss.backward()

            step = optimizer.step()

            #update epsilon
            if eps > eps_min:
                eps *= eps_decay
                
        #new state
        state = new_state
    print(score)
    print(list(zip(*np.unique(actions_dist, return_counts=True))))
    scores.append(score)

  0%|          | 0/50 [00:00<?, ?it/s]

/home/beast/.local/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


-379
[(0, 27), (1, 6), (2, 31)]
-511
[(0, 11), (1, 9), (2, 63)]
-429
[(0, 2), (3, 6), (4, 89)]
-446
[(0, 1), (4, 98)]
-380
[(0, 14), (1, 23), (2, 14), (3, 2), (4, 47)]
-420
[(0, 2), (1, 8), (2, 17), (3, 54), (4, 18)]
-385
[(0, 18), (1, 15), (2, 22), (3, 1), (4, 45)]
-310
[(0, 23), (1, 20), (2, 26), (3, 7), (4, 25)]
-375
[(0, 10), (1, 2), (2, 25), (3, 26), (4, 37)]
-377
[(0, 9), (1, 9), (2, 30), (3, 28), (4, 25)]
-376
[(0, 3), (1, 4), (2, 23), (3, 27), (4, 43)]
-392
[(0, 13), (1, 7), (2, 37), (3, 11), (4, 33)]
-429
[(0, 17), (1, 2), (2, 46), (3, 9), (4, 26)]
-412
[(0, 15), (1, 4), (2, 43), (3, 22), (4, 15)]
-361
[(0, 13), (1, 15), (2, 22), (3, 7), (4, 43)]
-308
[(0, 34), (1, 13), (2, 16), (3, 15), (4, 22)]
-370
[(0, 18), (1, 4), (2, 8), (3, 50), (4, 20)]
-383
[(0, 11), (1, 4), (2, 18), (3, 22), (4, 46)]
-385
[(0, 12), (1, 5), (2, 10), (3, 8), (4, 66)]
-379
[(0, 13), (1, 14), (2, 28), (3, 5), (4, 41)]
-377
[(0, 12), (1, 3), (2, 14), (3, 20), (4, 51)]
-319
[(0, 15), (1, 6), (2, 20), (3, 3

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

plt.plot(scores)
plt.ylabel('score')
plt.xlabel('episodes')
plt.title('Score of RL Agent over episodes')

reg = LinearRegression().fit(np.arange(len(scores)).reshape(-1, 1), np.array(scores).reshape(-1, 1))
y_pred = reg.predict(np.arange(len(scores)).reshape(-1, 1))
plt.plot(y_pred)

In [ ]:
scores = []
while len(scores) < 50:
    state = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action = select_action(network, state)[0]
        print(action)

        new_state, reward, done, _,_ = env.step(action)
        score += reward
        state = new_state
    scores.append(score)

In [ ]:
np.array(scores).mean()